In [7]:
!pip install -q langchain langchain-openai markitdown[all] langchain-text-splitters

In [3]:
from pathlib import Path

from markitdown import MarkItDown

In [5]:
TARGET_DIR = Path(r"/content/Электроника")  # Директория с учебным материалом
CONVERTED_DOCS_DIR = Path(r"/content/converted_docs")  # Документы сконвертиованные в Markdown
CONVERTED_DOCS_DIR.mkdir(exist_ok=True)

In [6]:
# Конвертация документов в Markdown

def convert_doc_to_md(path: Path) -> str:
  md = MarkItDown()
  result = md.convert(path)
  return result.text_content


for filepath in TARGET_DIR.iterdir():
  if filepath.suffix not in (".docx", ".pdf", ".pptx"):
    print("Not a target document, skip this")
    continue
  md_text = convert_doc_to_md(filepath)
  print(f"Document {filepath.name} converted to markdown")
  Path(CONVERTED_DOCS_DIR / f"{filepath.stem}.md").write_text(md_text, encoding="utf-8")
  print(f"Successfully saved to {filepath.stem}.md")

Document Метод указания к лабораторной работе Ознакомление с работой  Мультисим.docx converted to markdown
Successfully saved to Метод указания к лабораторной работе Ознакомление с работой  Мультисим.md
Document Метод указания автогенератор с мостом Вина_мультисим_2020.docx converted to markdown
Successfully saved to Метод указания автогенератор с мостом Вина_мультисим_2020.md
Document Метод указания Обратные связи  Мультисим.docx converted to markdown
Successfully saved to Метод указания Обратные связи  Мультисим.md
Document ФОС_Электроника _09.03.02_ИИП_22.docx converted to markdown
Successfully saved to ФОС_Электроника _09.03.02_ИИП_22.md
Document Метод указания усилительный каскад Мультисим 01_02_24.docx converted to markdown
Successfully saved to Метод указания усилительный каскад Мультисим 01_02_24.md
Document Лекция Биполярные транзисторы.pdf converted to markdown
Successfully saved to Лекция Биполярные транзисторы.md
Document Выпрямители и источники питания.pdf converted to mar

In [9]:
# Функции для разбиение документов на чанки

from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=150,
    length_function=len
)


def split_into_chunks(dir_path: Path) -> list[Document]:
  docs = []
  for filepath in dir_path.iterdir():
    if filepath.suffix != ".md":
      continue
    text = filepath.read_text(encoding="utf-8")
    doc = Document(page_content=text, metadata={"filename": filepath.stem})
    docs.append(doc)
  return text_splitter.split_documents(docs)


def format_chunk(doc: Document) -> str:
  return f"""**Текущий документ:** {doc.metadata.get("filename")}

  {doc.page_content}
  """

In [14]:
chunks = split_into_chunks(CONVERTED_DOCS_DIR)

print(f"Totall chunks count: {len(chunks)}")

formatted_chunks = [format_chunk(chunk) for chunk in chunks]

print("Chunk view example:")
print("=======================================")
print(f"{formatted_chunks[0][:500]}...")

Totall chunks count: 103
Chunk view example:
**Текущий документ:** Лекция Биполярные транзисторы

  Биполярные транзисторы

Биполярный транзистор – это система с двумя близко расположенными
p-n  переходами  с  тремя  внешними  выводами  (или  электродами)  :  эмиттер
(эммитирует или выбрасывает носители), коллектор (собирает эти носители),
база (основание транзистора).
Условная  плоскостная  структура  биполярного  транзистора  приведена  на
рисунке:

Источник Еэ имеет напряжение порядка 1 В (допустимое прямое напряжение
смещения),  при  э...


In [20]:
# Подключение к сервису Yandex Cloud - Foundation models

from google.colab import userdata

import openai

YANDEX_CLOUD_FOLDER_ID = userdata.get("YANDEX_CLOUD_FOLDER_ID")
YANDEX_CLOUD_APIKEY = userdata.get("YANDEX_CLOUD_APIKEY")
YANDEX_CLOUD_BASE_URL = "https://llm.api.cloud.yandex.net/v1"
SELECTED_MODEL = f"gpt://{YANDEX_CLOUD_FOLDER_ID}/yandexgpt/rc"  # YandexGPT Pro 5.1

client = openai.OpenAI(
    api_key=YANDEX_CLOUD_APIKEY,
    base_url="https://llm.api.cloud.yandex.net/v1",
    project=YANDEX_CLOUD_FOLDER_ID,
)

In [34]:
SYSTEM_MESSAGE = "Вы полезный эксперт по образовательным материалам и сфере образования, который извлекает информацию из документов."

PROMPT_TEMPLATE = """Твоя задача — проанализировать учебный материал образовательного курса и построить граф знаний, выделив ключевые концепции, их взаимосвязи и структуру курса.

0. ВСЕГДА ЗАВЕРШАЙ ВЫВОД ПОЛНОСТЬЮ. Не отправляй частичные ответы.

1. Сначала найди в тексте следующие типы сущностей и создай их в формате JSON. Для каждой сущности свойство `id` должно быть уникальной буквенно-цифровой строкой (латиница). Генерируй столько сущностей, сколько необходимо по типам ниже:

Типы сущностей:
- label:'УчебныйКурс', id:string, name:string, description:string // Основной курс или учебная программа; `id` формируется из названия курса в lowercase без пробелов и спецсимволов; `description` — краткое описание курса (1-2 предложения)
- label:'РазделКурса', id:string, name:string, order:integer // Крупный раздел или модуль курса; `order` — порядковый номер раздела в курсе
- label:'Тема', id:string, name:string, complexity:string // Учебная тема или концепция; `complexity` — уровень сложности: "базовый", "средний", "продвинутый"
- label:'Термин', id:string, name:string, definition:string // Ключевой термин или определение; `definition` — краткое определение (1 предложение)
- label:'Формула', id:string, name:string, expression:string // Математическая формула, уравнение, закон; `expression` — сама формула в текстовом виде
- label:'Принцип', id:string, name:string, description:string // Принцип, закон, правило, теорема; `description` — краткое описание принципа
- label:'Компонент', id:string, name:string, type:string // Физический компонент, устройство, элемент; `type` — тип компонента (например: "резистор", "транзистор", "микросхема")
- label:'Метод', id:string, name:string, application:string // Метод, методика, подход к решению; `application` — область применения метода
- label:'Пример', id:string, name:string, context:string // Практический пример, кейс, задача; `context` — контекст примера

2. Затем сгенерируй связи между сущностями в виде троек: голова, связь, хвост. Для ссылки на сущности используй их свойства `id`. Связь указывай в квадратных скобках. Генерируй столько связей, сколько необходимо:

Типы связей:
- курс|ВКЛЮЧАЕТ_РАЗДЕЛ|раздел
- раздел|РАССМАТРИВАЕТ_ТЕМУ|тема
- тема|СОДЕРЖИТ_ТЕРМИН|термин
- тема|ИСПОЛЬЗУЕТ_ФОРМУЛУ|формула
- тема|ОСНОВАНА_НА_ПРИНЦИПЕ|принцип
- принцип|ОПИСЫВАЕТ_КОМПОНЕНТ|компонент
- тема|ПРИМЕНЯЕТ_МЕТОД|метод
- метод|ПРОДЕМОНСТРИРОВАН_НА|пример
- тема|ПРОДЕМОНСТРИРОВАНА_НА|пример
- термин|ОПРЕДЕЛЯЕТ|компонент
- принцип|ВЫРАЖАЕТСЯ_ФОРМУЛОЙ|формула
- термин|СВЯЗАН_С|термин
- тема|ТРЕБУЕТ_ЗНАНИЯ|тема (для предварительных требований)
- формула|ЯВЛЯЕТСЯ_ЧАСТЬЮ|формула
- компонент|ИСПОЛЬЗУЕТСЯ_В|компонент

3. Дополнительные правила:
- Если в материале есть явная иерархия (часть-целое), отрази её соответствующими связями
- Если концепция имеет предварительные требования, укажи связь "ТРЕБУЕТ_ЗНАНИЯ"
- Для сложных концепций создавай несколько связей разного типа
- Если термин имеет синонимы или связанные понятия, используй связь "СВЯЗАН_С"
- Для формул, которые являются частями других формул, используй связь "ЯВЛЯЕТСЯ_ЧАСТЬЮ"

4. Выходные данные должны быть в следующем формате СТРОГО В ФОРМАТЕ JSON:

{{
    "entities": [
        {{"label": "УчебныйКурс", "id": "course_id", "name": "Название курса", "description": "Описание курса"}},
        {{"label": "Тема", "id": "topic_id", "name": "Название темы", "complexity": "базовый"}}
    ],
    "relationships": [
        "course_id|ВКЛЮЧАЕТ_РАЗДЕЛ|section_id",
        "topic_id|СОДЕРЖИТ_ТЕРМИН|term_id"
    ]
}}

ДОПОЛНИТЕЛЬНЫЕ ТРЕБОВАНИЯ К ФОРМАТАМ:
1. Для всех `id` используй camelCase (например: "bipolarTransistor", "emitterBaseCollector")
2. Для ID учебного курса используй только lowercase без пробелов и подчеркиваний (например: "electroniccomponents")
3. Все формулы записывай используя ТОЛЬКО латинские символы и стандартные математические операторы
4. Убедись, что каждая сущность имеет уникальный ID (даже если названия похожи)
5. Для компонентов указывай точный тип (например: "npnТранзистор", "pnpТранзистор", "резистор", "конденсатор")
6. Создавай больше связей типа СВЯЗАН_С для связанных терминов

ПРОВЕРЬ ПЕРЕД ОТВЕТОМ:
- Все ли ID в camelCase или lowercase?
- Все ли формулы на латинице?
- Есть ли связи СВЯЗАН_С для связанных терминов?
- Уникальны ли все ID?

Учебный материал:
{educational_content}"""

In [43]:
import json
from tqdm import tqdm


def call_llm(user_prompt: str, system_message: str) -> str:
    completion = client.chat.completions.create(
        model=SELECTED_MODEL,
        max_tokens=15000,
        temperature=0,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_prompt},
        ],
    )
    return completion.choices[0].message.content


def extract_entities_relationships(chunks: list[str]) -> list[dict]:
  results = []
  for chunk in tqdm(chunks):
      print(f"Extracting entities and relationships for {chunk[:100]}...")
      try:
        user_prompt = PROMPT_TEMPLATE.format(educational_content=chunk)
        result = call_llm(user_prompt, SYSTEM_MESSAGE)
        results.append(json.loads(result.replace("```", "")))
      except Exception as e:
        print(f"Error processing: {e}")
  return results

In [38]:
!pip install -q pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.4 MB/s eta 0:00:00


In [39]:
from pyvis.network import Network
import pandas as pd

def visualize_graph_pyvis(results, output_file="knowledge_graph.html"):
    """
    Интерактивная визуализация с PyVis
    """
    net = Network(height="800px", width="100%", bgcolor="#222222", font_color="white",
                  directed=True, notebook=False)

    # Настройки физики графа
    net.barnes_hut(gravity=-80000, central_gravity=0.3, spring_length=250, spring_strength=0.001)

    # Типы узлов и их стили
    node_styles = {
        'УчебныйКурс': {'color': '#FF6B6B', 'shape': 'star', 'size': 40},
        'РазделКурса': {'color': '#4ECDC4', 'shape': 'triangle', 'size': 35},
        'Тема': {'color': '#45B7D1', 'shape': 'dot', 'size': 30},
        'Термин': {'color': '#96CEB4', 'shape': 'square', 'size': 25},
        'Формула': {'color': '#FFEAA7', 'shape': 'diamond', 'size': 25},
        'Принцип': {'color': '#DDA0DD', 'shape': 'triangleDown', 'size': 30},
        'Компонент': {'color': '#F8B195', 'shape': 'box', 'size': 25},
        'Метод': {'color': '#6A5ACD', 'shape': 'hexagon', 'size': 30},
        'Пример': {'color': '#98D8C8', 'shape': 'circle', 'size': 20}
    }

    # Собираем данные
    all_entities = []
    all_relationships = []

    for result in results:
        all_entities.extend(result['entities'])
        all_relationships.extend(result['relationships'])

    # Добавляем узлы
    added_nodes = set()

    for entity in all_entities:
        node_id = entity['id']
        if node_id not in added_nodes:
            label = entity['label']
            style = node_styles.get(label, {'color': '#CCCCCC', 'shape': 'dot', 'size': 20})

            # Создаем всплывающую подсказку
            title = f"""
            <b>{entity['name']}</b><br>
            Тип: {label}<br>
            ID: {node_id}<br>
            """
            if 'description' in entity:
                title += f"Описание: {entity['description']}<br>"
            if 'complexity' in entity:
                title += f"Сложность: {entity['complexity']}<br>"
            if 'type' in entity:
                title += f"Тип: {entity['type']}<br>"

            net.add_node(node_id,
                        label=entity['name'][:20],  # Обрезаем длинные названия
                        title=title,
                        color=style['color'],
                        shape=style['shape'],
                        size=style['size'])
            added_nodes.add(node_id)

    # Добавляем связи
    for rel in all_relationships:
        if '|' in rel:
            try:
                head, relation, tail = rel.split('|')
                if head in added_nodes and tail in added_nodes:
                    net.add_edge(head, tail,
                                title=relation,
                                label=relation[:15],  # Обрезаем длинные названия связей
                                color='rgba(200, 200, 200, 0.7)',
                                width=2,
                                arrows='to')
            except:
                continue

    # Настраиваем отображение
    net.set_options("""
    var options = {
        "nodes": {
            "font": {
                "size": 14,
                "face": "Tahoma"
            }
        },
        "edges": {
            "color": {
                "inherit": true
            },
            "smooth": {
                "type": "continuous"
            },
            "font": {
                "size": 10,
                "face": "Tahoma",
                "align": "middle"
            }
        },
        "physics": {
            "forceAtlas2Based": {
                "gravitationalConstant": -50,
                "centralGravity": 0.01,
                "springLength": 200,
                "springConstant": 0.08
            },
            "maxVelocity": 50,
            "solver": "forceAtlas2Based",
            "timestep": 0.35,
            "stabilization": {
                "enabled": true,
                "iterations": 1000
            }
        },
        "interaction": {
            "hover": true,
            "tooltipDelay": 200
        }
    }
    """)

    # Сохраняем в HTML
    net.save_graph(output_file)
    print(f"Интерактивный граф сохранен в {output_file}")

    # Открываем в браузере
    import webbrowser
    webbrowser.open(output_file)

    return net

In [44]:
# Использование
results = extract_entities_relationships(formatted_chunks[:50])
net = visualize_graph_pyvis(results, "Граф_знаний_по_дисциплине_электроника.html")

  0%|          | 0/50 [00:00<?, ?it/s]

Extracting entities and relationships for **Текущий документ:** Лекция Биполярные транзисторы

  Биполярные транзисторы

Биполярный транзистор...


  2%|▏         | 1/50 [00:08<07:16,  8.90s/it]

Extracting entities and relationships for **Текущий документ:** Лекция Биполярные транзисторы

  тока (или тока основных носителей)  через пер...


  4%|▍         | 2/50 [00:21<08:50, 11.05s/it]

...

Extracting entities and relationships for **Текущий документ:** Метод указания Обратные связи  Мультисим

  Рисунок 12. Вид схемы усилительног...


 98%|█████████▊| 49/50 [06:24<00:07,  7.37s/it]

Extracting entities and relationships for **Текущий документ:** Метод указания Обратные связи  Мультисим

  В. Определить значения граничных ч...


100%|██████████| 50/50 [06:30<00:00,  7.80s/it]

Интерактивный граф сохранен в Граф_знаний_по_дисциплине_электроника.html


In [45]:
import plotly.graph_objects as go
import plotly.offline as pyo
import networkx as nx

def visualize_graph_plotly_3d(results, output_file="3d_graph.html"):
    """
    3D визуализация графа с Plotly
    """
    # Создаем граф NetworkX
    G = nx.Graph()

    # Собираем данные
    all_entities = []
    all_relationships = []

    for result in results:
        all_entities.extend(result['entities'])
        all_relationships.extend(result['relationships'])

    # Добавляем узлы
    node_labels = {}
    node_types = {}
    node_texts = {}

    for entity in all_entities:
        node_id = entity['id']
        G.add_node(node_id)
        node_labels[node_id] = entity['name']
        node_types[node_id] = entity['label']

        # Создаем текст для всплывающей подсказки
        text = f"<b>{entity['name']}</b><br>Тип: {entity['label']}<br>ID: {node_id}"
        if 'description' in entity:
            text += f"<br>Описание: {entity['description']}"
        node_texts[node_id] = text

    # Добавляем связи
    for rel in all_relationships:
        if '|' in rel:
            try:
                head, relation, tail = rel.split('|')
                if head in node_labels and tail in node_labels:
                    G.add_edge(head, tail, label=relation)
            except:
                continue

    # Создаем 3D позиции
    pos = nx.spring_layout(G, dim=3, k=2, iterations=50)

    # Подготавливаем данные для Plotly
    edge_trace = []
    for edge in G.edges():
        x0, y0, z0 = pos[edge[0]]
        x1, y1, z1 = pos[edge[1]]

        edge_trace.append(go.Scatter3d(
            x=[x0, x1, None],
            y=[y0, y1, None],
            z=[z0, z1, None],
            mode='lines',
            line=dict(width=1, color='#888'),
            hoverinfo='none'
        ))

    # Цвета по типам
    type_colors = {
        'УчебныйКурс': '#FF0000',
        'РазделКурса': '#00FF00',
        'Тема': '#0000FF',
        'Термин': '#FFFF00',
        'Формула': '#FF00FF',
        'Принцип': '#00FFFF',
        'Компонент': '#FFA500',
        'Метод': '#800080',
        'Пример': '#008080'
    }

    # Создаем следы для узлов каждого типа
    node_traces = []
    for node_type in type_colors:
        node_ids = [n for n in G.nodes() if node_types.get(n) == node_type]
        if node_ids:
            x = [pos[node][0] for node in node_ids]
            y = [pos[node][1] for node in node_ids]
            z = [pos[node][2] for node in node_ids]
            texts = [node_texts[node] for node in node_ids]
            labels = [node_labels[node] for node in node_ids]

            trace = go.Scatter3d(
                x=x, y=y, z=z,
                mode='markers+text',
                name=node_type,
                marker=dict(
                    size=10,
                    color=type_colors[node_type],
                    line=dict(width=2, color='white')
                ),
                text=labels,
                textposition="top center",
                hoverinfo='text',
                hovertext=texts
            )
            node_traces.append(trace)

    # Создаем фигуру
    fig = go.Figure(data=edge_trace + node_traces)

    # Настраиваем layout
    fig.update_layout(
        title='3D Граф знаний образовательного курса',
        scene=dict(
            xaxis=dict(showbackground=False, showticklabels=False, title=''),
            yaxis=dict(showbackground=False, showticklabels=False, title=''),
            zaxis=dict(showbackground=False, showticklabels=False, title=''),
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.5)
            )
        ),
        showlegend=True,
        width=1200,
        height=800
    )

    # Сохраняем и показываем
    fig.write_html(output_file)
    print(f"3D граф сохранен в {output_file}")
    fig.show()

    return fig

# Использование
fig = visualize_graph_plotly_3d(results)

3D граф сохранен в 3d_graph.html


In [46]:
with open(
    "Граф_знаний_по_дисциплине_электроника.json", "w", encoding="utf-8"
  ) as file:
  json.dump(results, file, ensure_ascii=False, indent=2)